In [35]:
LANDUSE_NDVI_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/neighborhoods_with_ndvi_numerical.geojson"
OSM_BUILDINGS_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/Taipei_Buildings_fulldata.geojson"
OSM_ROADS_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/taipei_segments_cleaned_verified.geoparquet"
OSM_TREES_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/taipei_land.geoparquet"
OSM_TRANSIT_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/taipei_infrastructure.geoparquet"
URBAN_MASTERPLAN_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/Taipei_urban_masterplan.geojson"
SUBGRAPH_DIR = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/subgraphs"
CHECKPOINT_DIR = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/checkpoints"

In [36]:
import geopandas as gpd
import cudf
import cuspatial
from shapely.geometry import Point
import pandas as pd
import os
from tqdm import tqdm
import cugraph
import pickle
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
import numpy as np
from keplergl import KeplerGl
import json

# Define actual file paths (customize these based on your system)
# Example assumes a project directory in your home folder: /home/your_username/project_folder/
BASE_DIR = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data"  # Replace "your_username" with your actual username

LANDUSE_NDVI_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/neighborhoods_with_ndvi_numerical.geojson"
OSM_BUILDINGS_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/Taipei_Buildings_fulldata.geojson"
OSM_ROADS_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/taipei_segments_cleaned_verified.geoparquet"
OSM_TREES_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/taipei_land.geoparquet"
OSM_TRANSIT_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/taipei_infrastructure.geoparquet"
URBAN_MASTERPLAN_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/Taipei_urban_masterplan.geojson"
SUBGRAPH_DIR = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/subgraphs"
CHECKPOINT_DIR = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/checkpoints"

# Ensure output directories exist
os.makedirs(SUBGRAPH_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Replace with your actual Mapbox access token
MAPBOX_ACCESS_TOKEN = "your_actual_mapbox_access_token"  # Get this from mapbox.com

### Helper Functions
def validate_geometries(gdf, name):
    """Validate and filter out invalid geometries in a GeoDataFrame."""
    print(f"Validating geometries for {name}...")
    gdf = gdf[gdf.geometry.is_valid].reset_index(drop=True)
    print(f"{name} after validation: {len(gdf)} rows")
    return gdf

### Stage 1: Data Loading and Preprocessing
def load_and_prepare_data():
    """Load and preprocess urban data for graph construction."""
    print("Stage 1: Loading and preparing data...")
    with tqdm(total=6, desc="Loading files") as pbar:
        neighborhoods_gdf = gpd.read_file(LANDUSE_NDVI_PATH, encoding='utf-8-sig')
        neighborhoods_gdf = validate_geometries(neighborhoods_gdf, "neighborhoods")
        pbar.update(1)

        buildings_gdf = gpd.read_file(OSM_BUILDINGS_PATH, encoding='utf-8-sig')
        buildings_gdf = validate_geometries(buildings_gdf, "buildings")
        if 'area_m2' not in buildings_gdf.columns:
            print("Computing area_m2 for buildings...")
            buildings_gdf = buildings_gdf.to_crs('EPSG:3826')  # Projected CRS for accurate area
            buildings_gdf['area_m2'] = buildings_gdf.geometry.area
            buildings_gdf = buildings_gdf.to_crs('EPSG:4326')  # Back to WGS84 for visualization
        pbar.update(1)

        roads_gdf = gpd.read_parquet(OSM_ROADS_PATH)
        roads_gdf = validate_geometries(roads_gdf, "roads")
        if roads_gdf.crs.is_geographic:
            print("Reprojecting roads_gdf to EPSG:3826 for length calculations...")
            roads_gdf = roads_gdf.to_crs('EPSG:3826')
        roads_gdf['length_m'] = roads_gdf.geometry.length
        pbar.update(1)

        trees_gdf = gpd.read_parquet(OSM_TREES_PATH)
        trees_gdf = trees_gdf[trees_gdf['subtype'] == 'tree']
        trees_gdf = validate_geometries(trees_gdf, "trees")
        pbar.update(1)

        transit_gdf = gpd.read_parquet(OSM_TRANSIT_PATH)
        transit_gdf = transit_gdf[transit_gdf['class'].isin(['stop_position', 'bus_stop'])]
        transit_gdf = transit_gdf[transit_gdf.geometry.geom_type == 'Point']
        transit_gdf = validate_geometries(transit_gdf, "transit")
        pbar.update(1)

        urban_masterplan_gdf = gpd.read_file(URBAN_MASTERPLAN_PATH)
        urban_masterplan_gdf = validate_geometries(urban_masterplan_gdf, "urban_masterplan")
        pbar.update(1)

    neighborhoods_geo = cuspatial.from_geopandas(neighborhoods_gdf['geometry'])
    trees_geo = cuspatial.from_geopandas(trees_gdf['geometry'])
    transit_geo = cuspatial.from_geopandas(transit_gdf['geometry'])

    neighborhoods_gdf['tree_count'] = 0
    neighborhoods_gdf['transit_count'] = 0
    for idx in range(len(neighborhoods_gdf)):
        neighborhood_geom = neighborhoods_geo.iloc[idx]
        if neighborhood_geom.is_valid:
            neighborhood_geo_series = cuspatial.GeoSeries([neighborhood_geom])
            neighborhoods_gdf.at[idx, 'tree_count'] = cuspatial.point_in_polygon(trees_geo, neighborhood_geo_series).sum().values[0]
            neighborhoods_gdf.at[idx, 'transit_count'] = cuspatial.point_in_polygon(transit_geo, neighborhood_geo_series).sum().values[0]
        else:
            print(f"Warning: Invalid geometry at index {idx}")

    return {
        'neighborhoods': neighborhoods_gdf,
        'buildings': buildings_gdf,
        'roads': roads_gdf,
        'trees': trees_gdf,
        'transit': transit_gdf,
        'urban_masterplan': urban_masterplan_gdf
    }

### Stage 2: Graph Construction
def build_graph(data):
    """Construct subgraphs for each neighborhood with nodes and edges."""
    print("Stage 2: Building graph network...")
    subgraphs = {}
    road_network_nodes = []

    neighborhoods_gdf = data['neighborhoods']
    buildings_gdf = data['buildings']
    roads_gdf = data['roads']
    trees_gdf = data['trees']
    transit_gdf = data['transit']

    # Define all feature columns to be included in every node
    feature_cols = [
        'land_use_residential_percent',
        'land_use_commercial_percent',
        'land_use_education_percent',
        'ndvi_mean',
        'tree_count',
        'transit_count',
        'area_m2',
        'length_m'
    ]
    default_node_data = {col: 0.0 for col in feature_cols}

    for idx in tqdm(range(len(neighborhoods_gdf)), desc="Building subgraphs"):
        lie_name = neighborhoods_gdf['LIE_NAME'].iloc[idx]
        subgraph_path = os.path.join(SUBGRAPH_DIR, f"subgraph_{lie_name}.pkl")

        if os.path.exists(subgraph_path):
            with open(subgraph_path, 'rb') as f:
                subgraphs[lie_name] = pickle.load(f)
            continue

        all_nodes = []
        all_edges = []

        # Neighborhood node
        node_id = f"neighborhood_{lie_name}"
        node_data = {
            'vertex': node_id,
            'type': 'neighborhood',
            'lie_name': lie_name,
            'population': neighborhoods_gdf['2024population'].iloc[idx],
            **default_node_data
        }
        node_data.update({
            'land_use_residential_percent': neighborhoods_gdf['land_use_residential_percent'].iloc[idx],
            'land_use_commercial_percent': neighborhoods_gdf['land_use_commercial_percent'].iloc[idx],
            'land_use_education_percent': neighborhoods_gdf['land_use_education_percent'].iloc[idx],
            'ndvi_mean': neighborhoods_gdf['ndvi_mean'].iloc[idx],
            'tree_count': neighborhoods_gdf['tree_count'].iloc[idx],
            'transit_count': neighborhoods_gdf['transit_count'].iloc[idx]
        })
        all_nodes.append(node_data)

        # Spatial buffer and filtering
        buffer_distance = 200  # meters
        neigh_geom = neighborhoods_gdf.geometry.iloc[idx]
        neigh_buffer = neigh_geom.buffer(buffer_distance)

        relevant_buildings = buildings_gdf[buildings_gdf.geometry.within(neigh_buffer)]
        relevant_roads = roads_gdf[roads_gdf.geometry.intersects(neigh_buffer)]
        relevant_trees = trees_gdf[trees_gdf.geometry.within(neigh_buffer)]
        relevant_transit = transit_gdf[transit_gdf.geometry.within(neigh_buffer)]

        # Building nodes
        for b_idx, building in relevant_buildings.iterrows():
            node_id = f"building_{b_idx}"
            node_data = {
                'vertex': node_id,
                'type': 'building',
                'building_type': building['building'],
                **default_node_data
            }
            node_data['area_m2'] = building['area_m2']
            all_nodes.append(node_data)

        # Road nodes
        road_points = []
        for r_idx, road in relevant_roads.iterrows():
            geom = road.geometry
            if geom.geom_type == 'LineString':
                road_points.extend([
                    (f"road_start_{r_idx}", Point(geom.coords[0])),
                    (f"road_end_{r_idx}", Point(geom.coords[-1]))
                ])

        for node_id, geom in road_points:
            r_idx = int(node_id.split('_')[2])
            road_data = {
                'vertex': node_id,
                'type': 'road',
                'road_class': roads_gdf['class'].iloc[r_idx],
                **default_node_data
            }
            road_data['length_m'] = roads_gdf['length_m'].iloc[r_idx]
            all_nodes.append(road_data)
            road_network_nodes.append(road_data)

        # Tree nodes
        for t_idx, tree in relevant_trees.iterrows():
            node_id = f"tree_{t_idx}"
            all_nodes.append({
                'vertex': node_id,
                'type': 'tree',
                **default_node_data
            })

        # Transit nodes
        for t_idx, transit in relevant_transit.iterrows():
            node_id = f"transit_{t_idx}"
            all_nodes.append({
                'vertex': node_id,
                'type': 'transit',
                'class': transit['class'],
                **default_node_data
            })

        # Edge creation
        for b_idx, building in relevant_buildings.iterrows():
            building_geom = building.geometry
            distance = neigh_geom.distance(building_geom)
            if distance < 100:
                all_edges.append({'src': f"neighborhood_{lie_name}", 'dst': f"building_{b_idx}", 'weight': distance})

        for b_idx, building in relevant_buildings.iterrows():
            building_geom = building.geometry
            for r_idx, road in relevant_roads.iterrows():
                road_geom = road.geometry
                distance = building_geom.distance(road_geom)
                if distance < 50:
                    all_edges.append({'src': f"building_{b_idx}", 'dst': f"road_start_{r_idx}", 'weight': distance})
                    all_edges.append({'src': f"building_{b_idx}", 'dst': f"road_end_{r_idx}", 'weight': distance})

        nodes_df = cudf.DataFrame(all_nodes)
        edges_df = cudf.DataFrame(all_edges) if all_edges else None
        
        # Debug: Print columns for this subgraph
        print(f"Columns in nodes_df for subgraph {lie_name}: {nodes_df.columns.tolist()}")

        subgraphs[lie_name] = {'nodes': nodes_df, 'edges': edges_df}

        with open(subgraph_path, 'wb') as f:
            pickle.dump(subgraphs[lie_name], f)

    return subgraphs, road_network_nodes

### Stage 3: Rule-Based Walkability Calculation
def compute_walkability_scores(nodes_df):
    """Calculate rule-based walkability scores for neighborhood nodes."""
    neighborhood_mask = nodes_df['type'] == 'neighborhood'
    neighborhood_df = nodes_df[neighborhood_mask].copy()

    if len(neighborhood_df) == 0:
        nodes_df['walkability_rule'] = 0.0
        return nodes_df

    residential = neighborhood_df['land_use_residential_percent']
    commercial = neighborhood_df['land_use_commercial_percent']
    education = neighborhood_df['land_use_education_percent']
    ndvi = neighborhood_df['ndvi_mean'].fillna(0.0)
    tree_count = neighborhood_df['tree_count']
    transit_count = neighborhood_df['transit_count']

    land_use_score = (residential * 0.4 + commercial * 0.3 + education * 0.2) / 100
    ndvi_score = ndvi * 0.5
    tree_score = (tree_count / 100).clip(upper=1.0) * 0.2
    transit_score = (transit_count / 20).clip(upper=1.0) * 0.2

    walkability = (land_use_score + ndvi_score * 0.4 + tree_score + transit_score).clip(upper=1.0)
    nodes_df['walkability_rule'] = 0.0
    nodes_df.loc[neighborhood_mask, 'walkability_rule'] = walkability

    return nodes_df

def calculate_walkability(subgraphs, neighborhoods_gdf):
    """Apply walkability scores to subgraphs and update neighborhoods_gdf."""
    print("Stage 3: Calculating rule-based walkability scores...")
    for lie_name, subgraph_data in tqdm(subgraphs.items(), desc="Calculating walkability"):
        nodes_df = subgraph_data['nodes']
        nodes_df = compute_walkability_scores(nodes_df)
        subgraph_data['nodes'] = nodes_df

        neighborhood_walkability = nodes_df[nodes_df['type'] == 'neighborhood']['walkability_rule']
        if not neighborhood_walkability.empty:
            neighborhoods_gdf.loc[neighborhoods_gdf['LIE_NAME'] == lie_name, 'walkability_rule'] = neighborhood_walkability.iloc[0]

    return subgraphs

### Stage 4: GNN Model with GAT and Validation
class GATWalkability(torch.nn.Module):
    """Graph Attention Network for walkability prediction."""
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GATWalkability, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=8, dropout=0.6)
        self.conv2 = GATConv(hidden_channels * 8, out_channels, heads=1, concat=False)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

def prepare_gnn_data(subgraphs):
    """Prepare data for GNN with feature normalization and additional features."""
    feature_cols = [
        'land_use_residential_percent',
        'land_use_commercial_percent',
        'land_use_education_percent',
        'ndvi_mean',
        'tree_count',
        'transit_count',
        'area_m2',
        'length_m'
    ]

    min_values = {col: float('inf') for col in feature_cols}
    max_values = {col: float('-inf') for col in feature_cols}

    for subgraph_data in subgraphs.values():
        nodes_df = subgraph_data['nodes'].to_pandas()
        for col in feature_cols:
            if col in nodes_df.columns:
                col_values = nodes_df[col].fillna(0)
                min_values[col] = min(min_values[col], col_values.min())
                max_values[col] = max(max_values[col], col_values.max())

    data_list = []
    for lie_name, subgraph_data in subgraphs.items():
        nodes_df = subgraph_data['nodes'].to_pandas()
        edges_df = subgraph_data['edges'].to_pandas() if subgraph_data['edges'] is not None else pd.DataFrame()

        # Debug: Print columns for this subgraph
        print(f"Columns in nodes_df for subgraph {lie_name} in prepare_gnn_data: {nodes_df.columns.tolist()}")

        node_mapping = {vertex: idx for idx, vertex in enumerate(nodes_df['vertex'])}
        nodes_df[feature_cols] = nodes_df[feature_cols].fillna(0)

        scaled_nodes_df = nodes_df[feature_cols].copy()
        for col in feature_cols:
            if max_values[col] > min_values[col]:
                scaled_nodes_df[col] = (scaled_nodes_df[col] - min_values[col]) / (max_values[col] - min_values[col])
            else:
                scaled_nodes_df[col] = 0

        x = torch.tensor(scaled_nodes_df.values, dtype=torch.float)

        if not edges_df.empty:
            edges_df['src_idx'] = edges_df['src'].map(node_mapping)
            edges_df['dst_idx'] = edges_df['dst'].map(node_mapping)
            edge_index = torch.tensor(edges_df[['src_idx', 'dst_idx']].values.T, dtype=torch.long)
        else:
            edge_index = torch.tensor([[], []], dtype=torch.long)

        y = torch.tensor(nodes_df['walkability_rule'].values, dtype=torch.float)
        data_list.append(Data(x=x, edge_index=edge_index, y=y))

    return data_list

def train_gnn_model(data_list):
    """Train the GAT model with validation."""
    print("Stage 4: Training GNN model...")
    model = GATWalkability(in_channels=8, hidden_channels=64, out_channels=1)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.MSELoss()

    train_data = data_list[:int(0.8 * len(data_list))]
    val_data = data_list[int(0.8 * len(data_list)):]

    for epoch in tqdm(range(200), desc="Training GNN"):
        model.train()
        train_loss = 0
        for data in train_data:
            optimizer.zero_grad()
            out = model(data.x, data.edge_index)
            loss = criterion(out, data.y.view(-1, 1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_data)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for data in val_data:
                out = model(data.x, data.edge_index)
                val_loss += criterion(out, data.y.view(-1, 1)).item()
        val_loss /= len(val_data)

        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    return model

def predict_walkability(subgraphs, model):
    """Predict walkability scores using the trained GAT model."""
    for lie_name, subgraph_data in subgraphs.items():
        nodes_df = subgraph_data['nodes'].to_pandas()
        data = prepare_gnn_data({lie_name: subgraph_data})[0]
        
        with torch.no_grad():
            pred = model(data.x, data.edge_index)
        nodes_df['walkability_gnn'] = pred.numpy().flatten()
        subgraph_data['nodes'] = cudf.from_pandas(nodes_df)
    return subgraphs

### Stage 5: Interactive Visualization
def create_interactive_map(subgraphs, data):
    """Generate an interactive Kepler.gl map with multiple urban data layers."""
    print("Stage 5: Generating interactive Kepler.gl map...")
    
    # Reproject all GeoDataFrames to EPSG:4326
    neighborhoods_gdf = data['neighborhoods'].to_crs('EPSG:4326')
    buildings_gdf = data['buildings'].to_crs('EPSG:4326')
    roads_gdf = data['roads'].to_crs('EPSG:4326')
    trees_gdf = data['trees'].to_crs('EPSG:4326')
    transit_gdf = data['transit'].to_crs('EPSG:4326')
    urban_masterplan_gdf = data['urban_masterplan'].to_crs('EPSG:4326')

    # Update neighborhoods with walkability scores from subgraphs
    for lie_name, subgraph_data in subgraphs.items():
        nodes_df = subgraph_data['nodes'].to_pandas()
        neigh_data = nodes_df[nodes_df['type'] == 'neighborhood'].iloc[0]
        neighborhoods_gdf.loc[neighborhoods_gdf['LIE_NAME'] == lie_name, 'walkability_rule'] = neigh_data['walkability_rule']
        neighborhoods_gdf.loc[neighborhoods_gdf['LIE_NAME'] == lie_name, 'walkability_gnn'] = neigh_data['walkability_gnn']

    # Prepare GeoJSON for each layer with relevant attributes
    neighborhoods_geojson = neighborhoods_gdf[['geometry', 'LIE_NAME', 'walkability_rule', 'walkability_gnn', 'tree_count', 'transit_count']].to_json()
    buildings_geojson = buildings_gdf[['geometry', 'building', 'area_m2']].to_json()
    roads_geojson = roads_gdf[['geometry', 'class', 'length_m']].to_json()
    trees_geojson = trees_gdf[['geometry']].to_json()
    transit_geojson = transit_gdf[['geometry', 'class']].to_json()
    urban_masterplan_geojson = urban_masterplan_gdf[['geometry', 'Category', 'Area']].to_json()

    # Initialize Kepler.gl map
    map_1 = KeplerGl(height=600, width=800, mapbox_api_access_token=MAPBOX_ACCESS_TOKEN)

    # Add all data layers
    map_1.add_data(data=neighborhoods_geojson, name="Neighborhoods")
    map_1.add_data(data=buildings_geojson, name="Buildings")
    map_1.add_data(data=roads_geojson, name="Roads")
    map_1.add_data(data=trees_geojson, name="Trees")
    map_1.add_data(data=transit_geojson, name="Transit")
    map_1.add_data(data=urban_masterplan_geojson, name="Urban Masterplan")

    # Load custom configuration if available
    CONFIG_PATH = os.path.join(BASE_DIR, "kepler.gl.json")
    if os.path.exists(CONFIG_PATH):
        with open(CONFIG_PATH, 'r') as f:
            custom_config = json.load(f)
        map_1.config = custom_config
        print(f"Loaded configuration from {CONFIG_PATH}")
    else:
        print("No configuration file found at kepler.gl.json. Default styling will be applied.")
        print("To customize, open the map in Kepler.gl, adjust layers, and export the configuration as kepler.gl.json in the project directory.")

    # Save the map
    output_path = os.path.join(BASE_DIR, "walkability_map.html")
    map_1.save_to_html(file_name=output_path)
    print(f"Interactive map saved at {output_path}")

### Main Execution
def main():
    """Execute the full walkability analysis pipeline."""
    try:
        data = load_and_prepare_data()
        subgraphs, road_network_nodes = build_graph(data)
        subgraphs = calculate_walkability(subgraphs, data['neighborhoods'])
        data_list = prepare_gnn_data(subgraphs)
        gnn_model = train_gnn_model(data_list)
        subgraphs = predict_walkability(subgraphs, gnn_model)
        create_interactive_map(subgraphs, data)
        print("Analysis completed successfully.")
    except Exception as e:
        print(f"Error during analysis: {e}")
        import traceback
        print(traceback.format_exc())

if __name__ == "__main__":
    main()

Stage 1: Loading and preparing data...


Loading files:  17%|█▋        | 1/6 [00:00<00:01,  3.27it/s]

Validating geometries for neighborhoods...
neighborhoods after validation: 456 rows
Validating geometries for buildings...
buildings after validation: 74306 rows
Computing area_m2 for buildings...


Loading files:  67%|██████▋   | 4/6 [00:05<00:02,  1.14s/it]

Validating geometries for roads...
roads after validation: 81444 rows
Validating geometries for trees...
trees after validation: 3399 rows


...

Training GNN: 100%|█████████▉| 199/200 [04:31<00:01,  1.28s/it]

Epoch 199, Train Loss: 0.0000, Validation Loss: 0.0001


Training GNN: 100%|██████████| 200/200 [04:32<00:00,  1.36s/it]

Epoch 200, Train Loss: 0.0000, Validation Loss: 0.0000
Columns in nodes_df for subgraph 湖田里 in prepare_gnn_data: ['vertex', 'type', 'lie_name', 'population', 'land_use_residential_percent', 'land_use_commercial_percent', 'land_use_education_percent', 'ndvi_mean', 'tree_count', 'transit_count', 'area_m2', 'length_m', 'road_class', 'walkability_rule']


Columns in nodes_df for subgraph 菁山里 in prepare_gnn_data: ['vertex', 'type', 'lie_name', 'population', 'land_use_residential_percent', 'land_use_commercial_percent', 'land_use_education_percent', 'ndvi_mean', 'tree_count', 'transit_count', 'area_m2', 'length_m', 'road_class', 'walkability_rule']
Columns in nodes_df for subgraph 大屯里 in prepare_gnn_data: ['vertex', 'type', 'lie_name', 'population', 'land_use_residential_percent', 'land_use_commercial_percent', 'land_use_education_percent', 'ndvi_mean', 'tree_count', 'transit_count', 'area_m2', 'length_m', 'road_class', 'walkability_rule']
Columns in nodes_df for subgraph 平等里 in prepare_gnn_data: ['vertex', 'type', 'lie_name', 'population', 'land_use_residential_percent', 'land_use_commercial_percent', 'land_use_education_percent', 'ndvi_mean', 'tree_count', 'transit_count', 'area_m2', 'length_m', 'road_class', 'walkability_rule']
Columns in nodes_df for subgraph 泉源里 in prepare_gnn_data: ['vertex', 'type', 'lie_name', 'population', 'land_